In [1]:
# !pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok huggingface_hub

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa",)



In [4]:
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse, JSONResponse
from PIL import Image
import torch
import io

from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

app = FastAPI()

@app.get("/")
async def form():
    return HTMLResponse("""
    <h2>Visual Question Answering Demo</h2>
    <form action="/predict" enctype="multipart/form-data" method="post">
        <input name="file" type="file" accept="image/*"><br><br>
        <input name="question" type="text" placeholder="Ask a question" size="40"><br><br>
        <input type="submit" value="Submit">
    </form>
    """)

@app.post("/predict")
async def predict(file: UploadFile = File(...), question: str = Form(...)):
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    inputs = processor(image, question, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    idx = outputs.logits.argmax(-1).item()
    answer = model.config.id2label[idx]

    return JSONResponse(content={"question": question, "answer": answer})


In [5]:
import numpy as np

# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt", padding=True)


In [6]:

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()


print("Predicted answer:", model.config.id2label[idx])

# TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [7]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [8]:
from typing import Union
from fastapi import FastAPI, UploadFile
import io
from PIL import Image

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.post("/ask")
def ask(text: str, image: UploadFile):
    content = image.file.read()

    image = Image.open(io.BytesIO(content))
    image = Image.open(image.file)

    result = model_pipeline(text, image)
    return {"answer": result}






In [9]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


ngrok.set_auth_token("2ynWK1oOeTMUMoXW5GhsR07Hy8Q_31UWTKwLoEZt7xTdzMYSm")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [34876]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://dd48-216-80-22-42.ngrok-free.app
INFO:     216.80.22.42:0 - "GET / HTTP/1.1" 200 OK
INFO:     216.80.22.42:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:58117 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:58117 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [34876]
